In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

import os
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed, Dropout, Flatten, Bidirectional, Concatenate, Reshape, Average, Add

from tensorflow.keras.models import Model
import tensorflow_hub as hub
import bert
FullTokenizer = bert.bert_tokenization.FullTokenizer
import math

Using TensorFlow backend.


Load the data

In [2]:
num_samples = 20000 # Number of samples to train on.

In [3]:
#read from dataset and split into input and targets
df = pd.read_csv("char_cleaned_data3.csv")
# df.fillna('', inplace=True)
df.dropna(subset = ["x1"], inplace=True)
df.dropna(subset = ["y1"], inplace=True)
df.head()

,x1,y1
0,what kind of phones do you guys have,i have a it is pretty great much better than w...
1,i have a it is pretty great much better than w...,does it really charge all the way in min
2,does it really charge all the way in min,pretty fast i have never it but it is under ha...
3,what kind of phones do you guys have,samsung galaxy j it is my first cell phone and...
4,samsung galaxy j it is my first cell phone and...,what do you think of it anything you do not like


In [4]:
print(len(df))
questions = df["x1"].tolist()[:num_samples]
answers = df["y1"].tolist()[:num_samples]

print(len(questions))
print(len(answers))

47365
20000
20000


In [5]:
max_seq_length = 128 
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=True)

In [37]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [38]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [350]:
vocab_file

b'C:\\Users\\charm\\AppData\\Local\\Temp\\tfhub_modules\\03d6fb3ce1605ad9e5e9ed5346b2fb9623ef4d3d\\assets\\vocab.txt'

In [39]:
def tokenize (s):
    stokens = tokenizer.tokenize(s)
    stokens =  ["[CLS]"] + stokens + ["[SEP]"]
    
    return stokens

input_ids = []
input_masks = []
input_segments = []

for line in questions:
    stokens = tokenize(line)
#     print(stokens)
    input_ids.append(get_ids(stokens, tokenizer, max_seq_length))
    input_masks.append(get_masks(stokens, max_seq_length))
    input_segments.append(get_segments(stokens, max_seq_length))

In [40]:
tokens = set()
for line in questions:
    t = tokenizer.tokenize(line)
    for i in t:
        tokens.add(i)

tokens = list(tokens)

In [41]:
START_TOKEN, END_TOKEN = [len(tokens)], [len(tokens) + 1]

In [42]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9) #close to -infinity

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [43]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads(attention)
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [44]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :]

In [45]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [46]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        # apply sin to even index in the array
        sines = tf.math.sin(angle_rads[:, 0::2])
        # apply cos to odd index in the array
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [47]:
#each encoder layer consists of: multi-head attention(with padding mask), 2 dense layers with dropout
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
          inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [247]:
#each encoder has an: input embedding, positional encoding, num_layers encoder layers
#embedding is summed with positional encoding, and then is passed to encoder layers 
#output of encoder goes to decoder

def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    
#     inputs = tf.keras.Input(shape=(None,), name="inputs")
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
    inputs = [input_word_ids, input_mask, segment_ids]
    _, embeddings = bert_layer(inputs)
#     embeddings = tf.reshape(embeddings, (-1, -1, 256))


    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

#     embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [32]:
#     input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
#                                        name="input_word_ids")
#     input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
#                                    name="input_mask")
#     segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
#                                     name="segment_ids")
#     inputs = [input_word_ids, input_mask, segment_ids]
#     _, embeddings = bert_layer(inputs)
#     embeddings = tf.reshape(embeddings, (-1, -1, 256))

In [34]:
# print (embeddings.shape)
# embeddings *= tf.math.sqrt(tf.cast(256, tf.float32))
# print(embeddings.shape)
# embeddings = PositionalEncoding( len(tokens) + 2, 256)(embeddings)
# print(embeddings.shape)

(None, None, 256)
(None, None, 256)
(None, None, 256)


In [248]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
          epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
        epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [249]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
        shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)


In [250]:
#lambda layer is an arbitrary layer so tensorflow functions can be used to create models 
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    #encoder inputs 
#     inputs = tf.keras.Input(shape=(None,), name="inputs")

    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
    
    inputs = [input_word_ids, input_mask, segment_ids]
    
    
    
    #decoder inputs 
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")
    
    #create padding mask for encoder inputs 
    # (batch_size, 1, 1, sequence length)
    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(input_word_ids)

    #decoder padding masks:
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(input_word_ids)

  #encoder 
    enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

  #decoder 
    dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [251]:
tf.keras.backend.clear_session()
VOCAB_SIZE = len(tokens) + 2

# Hyper-parameters
NUM_LAYERS = 4
D_MODEL = 768
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.2 

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [252]:
#since target sequences are padded, we apply a padding mask when calculating loss
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [253]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [254]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [255]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
________________________________________________________________________________________

In [60]:
# EPOCHS = 30
# model.fit(dataset, epochs=EPOCHS)

In [313]:
def evaluate(sentence):
    output = tf.expand_dims(START_TOKEN, 0)


    stokens = tokenizer.tokenize(sentence)
    stokens =  ["[CLS]"] + stokens + ["[SEP]"]

    ids = get_ids(stokens, tokenizer, max_seq_length)
    masks = get_masks(stokens, max_seq_length)
    segments = get_segments(stokens, max_seq_length)
                            
    ids = np.array(ids)
    masks = np.array(masks)
    segments = np.array(segments) 
    
    ids = tf.expand_dims(ids, axis=0)
    masks= tf.expand_dims(masks, axis=0)
    segments = tf.expand_dims(segments, axis = 0)
    
    input_sentence = [ids, masks, segments]
                            
#     for i in range(max_seq_length):
    for i in range(10):
        predictions = model(inputs=[input_sentence, output], training=False)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, END_TOKEN[0]):
            break

    # concatenated the predicted_id to the output which is given to the decoder
    # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

        
#     print( tf.squeeze(output, axis=0))
    return tf.squeeze(output, axis=0)


def predict(sentence):
        prediction = evaluate(sentence)

        predicted_sentence ='';
        for i in prediction:
            if i < len(tokens):
                predicted_sentence += ' ' + tokens[i]

        print('Input: {}'.format(sentence))
        print('Output: {}'.format(predicted_sentence))

        return predicted_sentence

testing begins here

In [340]:
sentence = "what is good"

output = tf.expand_dims(START_TOKEN, 0)


stokens = tokenizer.tokenize(sentence)
stokens =  ["[CLS]"] + stokens + ["[SEP]"]

ids = get_ids(stokens, tokenizer, max_seq_length)
masks = get_masks(stokens, max_seq_length)
segments = get_segments(stokens, max_seq_length)

In [341]:
ids = np.array(ids)
masks = np.array(masks)
segments = np.array(segments)
print (ids.shape)
print (masks.shape)
print (segments.shape)

(128,)
(128,)
(128,)


In [342]:
tokenize(sentence)

['[CLS]', 'what', 'is', 'good', '[SEP]']

In [343]:
print (ids)
print (masks)
print (segments)

[ 101 2054 2003 2204  102    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 

In [344]:
ids = tf.expand_dims(ids, axis=0)
masks= tf.expand_dims(masks, axis=0)
segments = tf.expand_dims(segments, axis = 0)
print (ids.shape)
print (masks.shape)
print (segments.shape)

(1, 128)
(1, 128)
(1, 128)


In [345]:
inp = [ids, masks, segments]

In [346]:
test = model(inputs=[inp, output]) 

In [349]:
# for i in range(max_seq_length):
for i in range(10):
    predictions = model(inputs=[inp, output], training=False)
    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]
    print(predictions)
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    for i in predicted_id:
        print (tokenizer.convert_ids_to_tokens(i))

#     # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, END_TOKEN[0]):
        break

# concatenated the predicted_id to the output which is given to the decoder
# as its input.
    output = tf.concat([output, predicted_id], axis=-1)

tf.Tensor(
[[[-0.2042409  -0.25284603  0.39858368 ...  0.6236401   0.36020344
    0.03334673]]], shape=(1, 1, 5972), dtype=float32)


TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [322]:
output

<tf.Tensor: shape=(1, 11), dtype=int32, numpy=array([[5970, 3087, 3087, 3087, 3087, 3087, 3087, 3087, 3087, 3087, 3087]])>

In [323]:
tokens[3087]

'com'

In [331]:
test = tf.squeeze(output, axis=0)

In [335]:
# predicted_sentence =''
# for i in test:
#     if i < len(tokens):
#         predicted_sentence += ' ' + tokens[i]
    

predicted_sentence = tokenizer.convert_ids_to_tokens(test)

TypeError: Tensor is unhashable. Instead, use tensor.ref() as the key.

In [306]:
predicted_sentence

' com com com com com com com com com com'

testing ends here

In [314]:
predict("this is a nice sentence")

tf.Tensor([5970  502  502  502  502  502  502  502  502  502  502], shape=(11,), dtype=int32)
Input: this is a nice sentence
Output:  backpack backpack backpack backpack backpack backpack backpack backpack backpack backpack


' backpack backpack backpack backpack backpack backpack backpack backpack backpack backpack'

In [308]:
output = predict("what is good")

Input: what is good
Output: 


In [137]:
text=[]
with open('testing.csv', mode='r') as f: 
  for line in f:
    text.append(line)

for line in text:
    sentence = predict(line)
    print ('')

Input: ï»¿i am an amazon warrior it means i shop on amazon a lot

Output: you are a part of the drinking age and a week like a lot

Input: how are you doing today

Output: i am not sure if that is how i feel hours drunk here

Input: i do not like you because of your face

Output: no but i have data

Input: what do you think about trump

Output: it is so

Input: i suppose you are smart can you make me a sandwich

Output: i am not even sure if our music is still onlinethank you

Input: can you do math on your own

Output: i have never had a girlfriend

Input: so i got a girlfriend today

Output: that is not a productive meeting then

Input: what kind of pizza do you like

Output: i am not sure if that is how i feel hours drunk haha

Input: i am working on a website project 

Output: i am not sure what is few fuck up how is your life going

Input: do you like chem homework

Output: thanks for understanding and not for the moment

Input: i ate leftover pizza from yesterday it taste bad

Ou